<a href="https://colab.research.google.com/github/Dikshuy/jax-MAML/blob/main/maml_omniglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import jax
import jax.numpy as jnp              
import optax                         
import numpy as np                   
import tensorflow_datasets as tfds    # TFDS for Omniglot
import tensorflow as tf
from jax import grad
from jax import vmap 
from functools import partial
from jax import jit 
from jax.experimental import stax 
from jax.experimental.stax import Conv, Dense, MaxPool, Relu, Flatten, Softmax, LogSoftmax, AvgPool, BatchNorm
import matplotlib.pyplot as plt 
from jax.experimental import optimizers
from jax.tree_util import tree_multimap
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

/usr/local/lib/python3.7/dist-packages/jax/experimental/stax.py:30: FutureWarning: jax.experimental.stax is deprecated, import jax.example_libraries.stax instead
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/jax/experimental/optimizers.py:30: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  FutureWarning)


In [3]:
rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)

In [4]:
learning_rate = 0.001
meta_step_size = 0.001

batch_size = 25

meta_iters = 2000
eval_iters = 5
inner_iters = 4

eval_interval = 1
train_shots = 20
shots = 5
classes = 5

In [5]:
class Dataset:
    # This class will facilitate the creation of a few-shot dataset
    # from the Omniglot dataset that can be sampled from quickly while also
    # allowing to create new labels at the same time.
    def __init__(self, training):
        # Download the tfrecord files containing the omniglot data and convert to a
        # dataset.
        split = "train" if training else "test"
        ds = tfds.load("omniglot", split=split, as_supervised=True, shuffle_files=False)
        # Iterate over the dataset to get each individual image and its class,
        # and put that data into a dictionary.
        self.data = {}

        def extraction(image, label):
            # This function will shrink the Omniglot images to the desired size,
            # scale pixel values and convert the RGB image to grayscale
            image = tf.image.convert_image_dtype(image, tf.float32)
            image = tf.image.rgb_to_grayscale(image)
            image = tf.image.resize(image, [28, 28])
            return image, label

        for image, label in ds.map(extraction):
            image = image.numpy()
            label = str(label.numpy())
            if label not in self.data:
                self.data[label] = []
            self.data[label].append(image)
        self.labels = list(self.data.keys())


    def get_mini_dataset(
        self, shots = shots, num_classes = classes
    ):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 28, 28, 1))
        test_labels = np.zeros(shape=(num_classes))
        test_images = np.zeros(shape=(num_classes, 28, 28, 1))

        # Get a random subset of labels from the entire label set.
        label_subset = random.choices(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            # label to create the test dataset.
            test_labels[class_idx] = class_idx
            images_to_split = random.choices(
                self.data[label_subset[class_idx]], k=shots + 1
            )
            test_images[class_idx] = images_to_split[-1]
            temp_images[
                class_idx * shots : (class_idx + 1) * shots
            ] = images_to_split[:-1]
        
        temp_images, temp_labels = shuffle(temp_images.astype(np.float32), temp_labels.astype(np.int32))
        
        support_set = {'images': temp_images, 'labels': temp_labels}
        query_set = {'images': test_images, 'labels': test_labels}
        
        return support_set, query_set

In [6]:
import urllib3

urllib3.disable_warnings()  # Disable SSL warnings that may happen during download.
train_dataset = Dataset(training=True)
test_dataset = Dataset(training=False)

In [7]:
net_init, net_apply = stax.serial(
    Conv(out_chan = 64, filter_shape = (3,3), strides = [1,1], padding = 'SAME'), BatchNorm(), Relu,
    Conv(out_chan = 64, filter_shape = (3,3), strides = [1,1], padding = 'SAME'), BatchNorm(), Relu,
    Conv(out_chan = 64, filter_shape = (3,3), strides = [1,1], padding = 'SAME'), BatchNorm(), Relu,
    Conv(out_chan = 64, filter_shape = (3,3), strides = [1,1], padding = 'SAME'), BatchNorm(), Relu,
    AvgPool((28, 28)),
    Flatten,
    Dense(classes),
)

in_shape = (-1, 28, 28, 1)
out_shape, net_params = net_init(rng, in_shape)

In [8]:
@jit
def loss(params, inputs, targets, flag = False):
    predictions = net_apply(params, inputs)
    loss_ = jnp.mean(optax.softmax_cross_entropy(predictions, jax.nn.one_hot(targets, num_classes=classes)))
    accuracy = jnp.mean(jnp.argmax(predictions, -1) == targets)
    if flag:
        return loss_, accuracy
    else:
        return loss_
    # return loss_

In [9]:
@jit
def inner_update(params, inputs, outputs, alpha = learning_rate):
    '''
    input:
    - params: model's parameters
    - inputs
    - targets: true label
    output
    - updated parameters
    '''
    grads = grad(loss)(params, inputs, outputs)
    grad_update_fn = lambda g, state: (state - alpha * g)
    return tree_multimap(grad_update_fn, grads, params)

In [10]:
@jit
def maml_loss(params, support_img, support_lab, query_img, query_lab, num_inner_loops=2, flag = True):
    '''
    input:
    - params: model's parameters
    - x1, y1: task's train set
    - x2, y2: task's test set
    output:
    - Loss after update parameters 1 time on the test set.
    '''
    params_updated = params
    for _ in range(num_inner_loops):
        params_updated = inner_update(params_updated, support_img, support_lab)
    total_loss, accuracy = loss(params_updated, query_img, query_lab, flag = True)
    if flag:
        return total_loss, accuracy
    else:
        return total_loss
    # total_loss = loss(params_updated, query_img, query_lab)
    # return total_loss

In [11]:
import numpy as onp
from jax.experimental import optimizers
from jax.tree_util import tree_multimap 

In [12]:
# Define optimizer
opt_init, opt_update, get_params = optimizers.adam(step_size=meta_step_size)
opt_state = opt_init(net_params)  

In [13]:
from tqdm import tqdm
#Define a step (using jit to improve speed)
@jit
def step(i, opt_state, support_img, support_lab, query_img, query_lab):
    '''
    input:
    - step number, opt_state (contains params)
    -x1, y1: train, x2, y2: test and get loss
    output:
    - new opt_state and loss
    '''
    # Get params from opt_state
    p = get_params(opt_state)
    # calculate gradient from maml_loss
    g = grad(maml_loss)(p, support_img, support_lab, query_img, query_lab)
    # pre-model update trial on task.
    l, acc = maml_loss(p, support_img, support_lab, query_img, query_lab, flag = True)
    if i is not None:
        return opt_update(i, g, opt_state), l, acc
    else:
        return l, acc
    # l = maml_loss(p, support_img, support_lab, query_img, query_lab)
    # return opt_update(i, g, opt_state), l

In [23]:
@jit
def maml_loss_batch(params, x1_b, y1_b, x2_b, y2_b, flag = False):
    '''
    input:
    - params
    - x1_b, y1_b, x2_b, y2_b: batches of sample task 
    output:
    - combined loss of the batch
    '''   
    return  onp.mean(vmap(partial(maml_loss, params))(x1_b, y1_b, x2_b, y2_b, flag=True)), None
    # total_loss, acc = maml_loss(params, x1_b, y1_b, x2_b, y2_b)
    # if flag:
    #     return  onp.mean(total_loss), onp.mean(acc)
    # else:
    #     return  onp.mean(vmap(partial(maml_loss, params))(x1_b, y1_b, x2_b, y2_b))

In [15]:
@jit
def batch_step(i, opt_state, x1_b, y1_b, x2_b, y2_b):
    p = get_params(opt_state)
    g = grad(maml_loss_batch)(p, x1_b, y1_b, x2_b, y2_b)
    # l = maml_loss_batch(p, x1_b, y1_b, x2_b, y2_b)
    l, acc = maml_loss_batch(p, x1_b, y1_b, x2_b, y2_b, flag = True)
    if i is not None:
        return opt_update(i, g, opt_state), l, acc
    else:
        return l, acc

In [24]:
TRAIN_STEPS = 5
maml_losses = []
maml_accuracy = []
for i in tqdm(range(TRAIN_STEPS)):
    # get x_support, y_support, x_query, y_query batch
    x_support_batch = []
    y_support_batch = []
    x_query_batch = []
    y_query_batch = []
    for j in range(batch_size):
        support, query = train_dataset.get_mini_dataset()
        x_support, y_support = support['images'], support['labels']
        x_query, y_query = query['images'], query['labels']
        x_support_batch.append(x_support)
        y_support_batch.append(y_support)
        x_query_batch.append(x_query)
        y_query_batch.append(y_query)
    x_support_batch = np.stack(x_support_batch)
    y_support_batch = np.stack(y_support_batch)
    x_query_batch = np.stack(x_query_batch)
    y_query_batch = np.stack(y_query_batch)
    # opt_state, l = batch_step(i, opt_state, x_support_batch, y_support_batch, x_query_batch, y_query_batch)
    opt_state, l, acc = batch_step(i, opt_state, x_support_batch, y_support_batch, x_query_batch, y_query_batch)
    maml_losses.append(l)
    maml_accuracy.append(acc)

net_params = get_params(opt_state)

  0%|          | 0/5 [00:00<?, ?it/s]


ValueError: ignored

In [ ]:
plt.plot(np.arange(TRAIN_STEPS), np.array(maml_losses))
maml_losses
# maml_accuracy